# 13F InfoTable with YearQuarter
This notebook reads Excel workbooks from `data/extracted_13F_HR/blackrock`, loads only the `InfoTable` sheet, and adds a `YearQuarter` column derived from the filename (e.g., `2024Q3`).


In [15]:
from pathlib import Path
import pandas as pd
import re

DATA_DIR = Path('../data/extracted_13F_HR/blackrock')
print('Data dir exists:', DATA_DIR.exists())
print('Sample files:', [p.name for p in sorted(DATA_DIR.glob('*.xlsx'))[:3]])


Data dir exists: True
Sample files: ['20240930.xlsx', '20241231.xlsx', '20250331.xlsx']


In [22]:
def year_quarter_from_filename(name: str):
    m = re.search(r'(\d{4})(\d{2})(\d{2})', name)
    if not m:
        return None
    year = int(m.group(1))
    month = int(m.group(2))
    q = (month - 1) // 3 + 1
    return f'{year}Q{q}'

files = sorted(DATA_DIR.glob('*.xlsx'))
frames = []
for f in files:
    yq = year_quarter_from_filename(f.name)
    df = None
    try:
        df = pd.read_excel(f, sheet_name='InfoTable', engine='openpyxl')
    except Exception as e:
        try:
            df = pd.read_excel(f, sheet_name='Information Table', engine='openpyxl')
        except Exception as e2:
            print(f'Skipping {f.name}: {e2}')
            continue
    df['YearQuarter'] = yq
    frames.append(df)

result = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
print('Rows:', len(result))
result.head()


Rows: 199316


,issuer_name,class_title,cusip,value_usd_quarter_end,shares_or_principal,shares_type,discretion,other_manager_seq,vote_sole,vote_shared,vote_none,YearQuarter
0,1 800 FLOWERS COM INC,CL A,68243Q106,1769262,223110,SH,SOLE,2.0,124034,0,99076,2024Q3
1,1 800 FLOWERS COM INC,CL A,68243Q106,761,96,SH,SOLE,4.0,96,0,0,2024Q3
2,1 800 FLOWERS COM INC,CL A,68243Q106,2990,377,SH,SOLE,7.0,377,0,0,2024Q3
3,1 800 FLOWERS COM INC,CL A,68243Q106,793222,100028,SH,SOLE,18.0,74838,0,25190,2024Q3
4,1 800 FLOWERS COM INC,CL A,68243Q106,5000087,630528,SH,SOLE,19.0,630528,0,0,2024Q3


In [25]:


# 1. Get the unique class titles as a list
unique_titles = result.class_title.unique().tolist()

# 2. Define the output file name
file_name = "unique_class_titles.txt"

# 3. Write the list to the text file
try:
    with open(file_name, 'w') as f:
        # Write each title on a new line
        for title in unique_titles:
            f.write(f"{title}\n")
    print(f"Successfully saved {len(unique_titles)} unique titles to **{file_name}**")

except Exception as e:
    print(f"An error occurred while writing the file: {e}")

Successfully saved 1255 unique titles to **unique_class_titles.txt**


In [24]:
result[result.class_title == 'ACACIA TCH COM']

,issuer_name,class_title,cusip,value_usd_quarter_end,shares_or_principal,shares_type,discretion,other_manager_seq,vote_sole,vote_shared,vote_none,YearQuarter
317,ACACIA RESH CORP,ACACIA TCH COM,003881307,46157,9905,SH,SOLE,2.0,1324,0,8581,2024Q3
318,ACACIA RESH CORP,ACACIA TCH COM,003881307,506826,108761,SH,SOLE,18.0,108761,0,0,2024Q3
319,ACACIA RESH CORP,ACACIA TCH COM,003881307,388528,83375,SH,SOLE,19.0,83375,0,0,2024Q3
320,ACACIA RESH CORP,ACACIA TCH COM,003881307,6383832,1369921,SH,SOLE,21.0,1369921,0,0,2024Q3
321,ACACIA RESH CORP,ACACIA TCH COM,003881307,4709293,1010578,SH,SOLE,22.0,983145,0,27433,2024Q3
322,ACACIA RESH CORP,ACACIA TCH COM,003881307,93233,20007,SH,SOLE,43.0,19438,0,569,2024Q3
48419,ACACIA RESH CORP,ACACIA TCH COM,003881307,60040,13834,SH,SOLE,2.0,1324,0,12510,2024Q4
48420,ACACIA RESH CORP,ACACIA TCH COM,003881307,463317,106755,SH,SOLE,18.0,106755,0,0,2024Q4
48421,ACACIA RESH CORP,ACACIA TCH COM,003881307,311438,71760,SH,SOLE,19.0,71760,0,0,2024Q4
48422,ACACIA RESH CORP,ACACIA TCH COM,003881307,5807519,1338138,SH,SOLE,21.0,1338138,0,0,2024Q4


In [18]:
result['share_price_quarter_end'] = result['value_usd_quarter_end']/result['shares_or_principal']

In [20]:
tmp1 = result[(result.class_title == "COM") & (result.share_price_quarter_end < 2)]
tmp1

,issuer_name,class_title,cusip,value_usd_quarter_end,shares_or_principal,shares_type,discretion,other_manager_seq,vote_sole,vote_shared,vote_none,YearQuarter,share_price_quarter_end
29,180 LIFE SCIENCES CORP,COM,68236V302,37,22,SH,SOLE,2.0,22,0,0,2024Q3,1.681818
30,180 LIFE SCIENCES CORP,COM,68236V302,905,542,SH,SOLE,22.0,542,0,0,2024Q3,1.669742
53,22ND CENTY GROUP INC,COM,90137F301,2004,9317,SH,SOLE,22.0,9317,0,0,2024Q3,0.215091
79,374WATER INC,COM,88583P104,9298,6837,SH,SOLE,2.0,2030,0,4807,2024Q3,1.359953
80,374WATER INC,COM,88583P104,206176,151600,SH,SOLE,18.0,151600,0,0,2024Q3,1.360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199011,ZENTALIS PHARMACEUTICALS INC,COM,98943L107,296,255,SH,SOLE,7.0,255,0,0,2025Q2,1.160784
199012,ZENTALIS PHARMACEUTICALS INC,COM,98943L107,50655,43668,SH,SOLE,18.0,39509,0,4159,2025Q2,1.160003
199013,ZENTALIS PHARMACEUTICALS INC,COM,98943L107,23112,19924,SH,SOLE,19.0,19924,0,0,2025Q2,1.160008
199014,ZENTALIS PHARMACEUTICALS INC,COM,98943L107,550033,474166,SH,SOLE,21.0,474166,0,0,2025Q2,1.160001


In [21]:
com_less_than_2_usd = tmp1.issuer_name.unique()
com_less_than_2_usd

array(['180 LIFE SCIENCES CORP', '22ND CENTY GROUP INC', '374WATER INC',
       'ACCURAY INC', 'ACLARIS THERAPEUTICS INC',
       'ACTINIUM PHARMACEUTICALS INC', 'ACURX PHARMACEUTICALS INC',
       'ADICET BIO INC', 'AGRIFY CORP', 'AIM IMMUNOTECH INC', 'AINOS INC',
       'AKEBIA THERAPEUTICS INC', 'AKOUSTIS TECHNOLOGIES INC',
       'ALLAKOS INC', 'ALLIED GAMING & ENTRTNMNT IN', 'ALLOVIR INC',
       'ALT5 SIGMA CORP', 'ALTO INGREDIENTS INC',
       'ALX ONCOLOGY HLDGS INC', 'AMMO INC', 'AN2 THERAPEUTICS INC',
       'ANEBULO PHARMACEUTICALS INC', 'APTEVO THERAPEUTICS INC',
       'APYX MEDICAL CORPORATION', 'AQUA METALS INC',
       'AQUABOUNTY TECHNOLOGIES INC', 'ATHIRA PHARMA INC',
       'ATLANTIC AMERN CORP', 'ATOSSA THERAPEUTICS INC',
       'AUTONOMIX MED INC', 'AWARE INC MASS', 'AYRO INC',
       'BALLARD PWR SYS INC NEW', 'BARK INC', 'BEYOND AIR INC',
       'BIGBEAR AI HLDGS INC', 'BIOATLA INC', 'BIODESIX INC',
       'BIOXCEL THERAPEUTICS INC', 'BITCOIN DEPOT INC',
       '

In [7]:
tmp2 = result[result.issuer_name.isin(com_less_than_2_usd)]

In [12]:
tmp2[tmp2.issuer_name == "FORIAN INC"]

,issuer_name,class_title,cusip,value_usd_thousands,shares_or_principal,shares_type,discretion,other_manager_seq,vote_sole,vote_shared,vote_none,YearQuarter,share_price
17531,FORIAN INC,COM,34630N106,8435,3905,SH,SOLE,18.0,3905,0,0,2024Q3,2.160051
17532,FORIAN INC,COM,34630N106,93480,43278,SH,SOLE,21.0,43278,0,0,2024Q3,2.159989
17533,FORIAN INC,COM,34630N106,91318,42277,SH,SOLE,22.0,42277,0,0,2024Q3,2.159992
17534,FORIAN INC,COM,34630N106,2,1,SH,SOLE,43.0,0,0,1,2024Q3,2.000000
66576,FORIAN INC,COM,34630N106,8044,3905,SH,SOLE,18.0,3905,0,0,2024Q4,2.059923
66577,FORIAN INC,COM,34630N106,89762,43574,SH,SOLE,21.0,43574,0,0,2024Q4,2.059990
66578,FORIAN INC,COM,34630N106,87091,42277,SH,SOLE,22.0,42277,0,0,2024Q4,2.060009
66579,FORIAN INC,COM,34630N106,2,1,SH,SOLE,43.0,0,0,1,2024Q4,2.000000
117176,FORIAN INC,COM,34630N106,7810,3905,SH,SOLE,18.0,3905,0,0,2025Q1,2.000000
117177,FORIAN INC,COM,34630N106,85372,42686,SH,SOLE,21.0,42686,0,0,2025Q1,2.000000


In [14]:
tmp4 = result[(result.share_price > 150)]